In [1]:
%matplotlib widget
from spectraclass.data.base import DataManager
from spectraclass.data.base import ModeDataManager
from spectraclass.data.spatial.tile.manager import TileManager
from spectraclass.application.controller import app, SpectraclassController
from spectraclass.model.labels import LabelsManager, lm
import os
from typing import List, Union, Tuple, Optional, Dict, Callable
host = "desktop" # "desktop" # "laptop"

Here we configure paths on the Jupyter server.  If these paths are not specified here then the default values,
    defined in server-side config files, for the project ("demo2") and data mode ("desis"), will be used.  You can
    choose whatever project names you want, they are used to save configurations and results for ongoing investigations.

In [2]:
dm: DataManager = DataManager.initialize( "demo2", 'aviris' )
if (host == "laptop"):
    dm.modal.data_dir = os.path.expanduser("~/Development/Data/Aviris/processed")
    dm.modal.cache_dir = os.path.expanduser("/Volumes/archive/Cache")
else:
    dm.modal.cache_dir = "/Volumes/Shared/Cache"
    dm.modal.data_dir = "/Users/tpmaxwel/Development/Data/Aviris"
    
dm.modal.image_names = [ "20170720t004130" ]
dm.proc_type = "cpu"
TileManager.block_size = 256     # 250
ModeDataManager.model_dims = 24  # 16
TileManager.block_index = [0,2]

SyntaxError: invalid syntax (2526866277.py, line 1)

Here we define a set of class names and associated colors to be used in the classification process.

In [ ]:
classes = [ ('Class-1', "cyan"),
            ('Class-2', "green"),
            ('Class-3', "magenta"),
            ('Class-4', "blue")]


Here we start up the Spectraclass GUI.

In [ ]:
dm.loadCurrentProject()
lm().setLabels( classes )
controller: SpectraclassController = app()
controller.gui()